In [1]:
from llama_index.llms.ollama import Ollama
from llama_index.llms.groq import Groq

class Generators:
    def __init__(self, model="llama-3.3-70b-versatile"):
        # self.llm = Ollama(model=model, temperature=0)
        self.llm = Groq(model=model, api_key="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", temperature=0)

    def get_llm(self):
        return self.llm

c:\Users\ASUS\anaconda3\envs\agent\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [2]:
import math

session = dict()

def multiply(a: float, b: float) -> float:
    """useful tool to Multiply two numbers and returns the product
    make sure that both numbers a, b are real numbers and if not try to convert them to real numbers
    args:
        a: float
        b: float    
    requires:
        only_real_numbers
    """
    return float(a) * float(b)

def add(a: float, b: float) -> float:
    """useful tool to Add two numbers and returns the sum, 
    make sure that both numbers a, b are real numbers and if not try to convert them to real numbers
    args:
        a: float
        b: float    

    requires:
        only_real_numbers
    """
    return float(a) + float(b)

def only_real_numbers(a: float) -> float:
    """ useful tool to verify if the number is real"""
    try: 
        float(a)
    except ValueError: 
        return f"{a} is not a real number so try converting it to a float and try again"

In [7]:
from llama_index.core.agent import FunctionCallingAgent
from llama_index.core.tools import FunctionTool

add_tool = FunctionTool.from_defaults(fn=add)
multiply_tool = FunctionTool.from_defaults(fn=multiply)
real_number_tool = FunctionTool.from_defaults(fn=only_real_numbers)

class AgentController:
    def __init__(self):        
        self.llm = Generators().get_llm()
        self.system_prompt = """
                                You are a math expert. You will only use the tools available to you.
                                IMPORTANT NOTE: You will ALWAYS evaluate the user's query and perfom query classification and provide three things:
                                tool_used, reasoning, answer
                                
                                like this:

                                Answer: answer
                                Tool Used: tool_name
                                Reasoning: reasoning for using the tool

                                An example:

                                Answer: 21.0
                                Tool Used: multiply
                                Reasoning: The tool was used to calculate the product of two numbers.
                                
                                
                                Solve the queries STEP by STEP and feel free to use the tools available to you and do not hallucinate or make assumptions.
                                """
        self.agent = self.get_agent()

    def get_agent(self):
        agent = FunctionCallingAgent.from_tools([multiply_tool, add_tool,
                                                 real_number_tool],
                                        llm=self.llm,verbose=True,
                                        system_prompt=self.system_prompt)
        return agent
    
    def chat(self, query: str):
        response_obj = self.agent.chat(query)
        return response_obj

In [8]:
agent = AgentController()
query = "what is 2+2"
agent.chat(query)

> Running step d363ed0f-4698-4837-92cc-f901f9a37b4f. Step input: what is 2+2
Added user message to memory: what is 2+2
=== Calling Function ===
Calling function: add with args: {"a": 2, "b": 2}
=== Function Output ===
4.0
> Running step 88146b13-79f2-4a97-9f9a-b22ab7ba2b3e. Step input: None
=== LLM Response ===
Answer: 4.0
Tool Used: add
Reasoning: The tool was used to calculate the sum of two numbers.


AgentChatResponse(response='Answer: 4.0\nTool Used: add\nReasoning: The tool was used to calculate the sum of two numbers.', sources=[ToolOutput(content='4.0', tool_name='add', raw_input={'args': (), 'kwargs': {'a': 2, 'b': 2}}, raw_output=4.0, is_error=False)], source_nodes=[], is_dummy_stream=False, metadata=None)